### Step 1: Install Dependencies

### Step 2: Setup and Imports

In [1]:
import os
import shutil
import numpy as np
import cv2
from pathlib import Path
from sklearn.model_selection import train_test_split
import albumentations as A
from ultralytics import YOLO
import yaml
from tqdm import tqdm


BASE_DIR = Path.cwd()
ORIGINAL_DATA = BASE_DIR / "2.v1i.yolov11"
SPLIT_DATA = BASE_DIR / "dataset_split"
AUGMENTED_DATA = BASE_DIR / "dataset_augmented"

### Step 3: Split Dataset (Train/Val/Test)

In [ ]:
def split_dataset(
    source_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15
):
    for split in ["train", "val", "test"]:
        (output_dir / split / "images").mkdir(parents=True, exist_ok=True)
        (output_dir / split / "labels").mkdir(parents=True, exist_ok=True)

    images_dir = source_dir / "train" / "images"
    labels_dir = source_dir / "train" / "labels"

    image_files = sorted(
        [f for f in os.listdir(images_dir) if f.endswith((".jpg", ".jpeg", ".png"))]
    )
    print(f"Total images: {len(image_files)}")

    train_val, test = train_test_split(
        image_files, test_size=test_ratio, random_state=42
    )

    train, val = train_test_split(
        train_val, train_size=train_ratio / (train_ratio + val_ratio), random_state=42
    )

    splits = {"train": train, "val": val, "test": test}

    for split, files in splits.items():
        for img_file in files:
            label_file = img_file.rsplit(".", 1)[0] + ".txt"

            src_img = images_dir / img_file
            dst_img = output_dir / split / "images" / img_file
            shutil.copy2(src_img, dst_img)

            src_label = labels_dir / label_file
            if src_label.exists():
                dst_label = output_dir / split / "labels" / label_file
                shutil.copy2(src_label, dst_label)

        print(f"{split.upper()}: {len(files)} images")

    return splits


if SPLIT_DATA.exists():
    shutil.rmtree(SPLIT_DATA)

splits = split_dataset(ORIGINAL_DATA, SPLIT_DATA)

Total images: 500
TRAIN: 350 images
VAL: 75 images
TEST: 75 images


### Step 4: Create data.yaml for YOLOv11

In [3]:
data_yaml = {
    "path": str(SPLIT_DATA),
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": 1,
    "names": ["fruit"],
}

yaml_path = SPLIT_DATA / "data.yaml"
with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print(yaml.dump(data_yaml))

names:
- fruit
nc: 1
path: /home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split
test: test/images
train: train/images
val: val/images



### Step 5: Train YOLOv11 on Original Dataset

In [4]:
model = YOLO("yolo11n.pt")
model_path = BASE_DIR / "yolo11n.pt"
model = YOLO(str(model_path))

results = model.train(
    data=str(yaml_path),
    epochs=50,
    imgsz=640,
    batch=16,
    patience=10,
    device=0,
    project=str(BASE_DIR / "runs"),
    name="yolov11_original",
    save=True,
    verbose=True,
)

Ultralytics 8.3.235 🚀 Python-3.10.18 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24219MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/home/usr4/Arsenii/sbor_data_pfur/lab_7/yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11_original5, nbs=64, nms=False, opset=None, o

### Step 6: Evaluate on Test Set

In [5]:
best_model = YOLO("runs/yolov11_original3/weights/best.pt")

test_results = best_model.val(
    data=str(yaml_path), split="test", imgsz=640, batch=16, device=0
)

test_results.results_dict

Ultralytics 8.3.235 🚀 Python-3.10.18 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24219MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2496.3±1098.8 MB/s, size: 18.1 KB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2496.3±1098.8 MB/s, size: 18.1 KB)
val: Scanning /home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split/test/labels... 75 images, 13 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 75/75 2.3Kit/s 0.0s
val: New cache created: /home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split/test/labels.cache
val: Scanning /home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split/test/labels... 75 images, 13 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 75/75 2.3Kit/s 0.0s
val: New cache created: /home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split/test/labels.cache
                 Class     Images  Instances      Box(P 

{'metrics/precision(B)': 0.0034222222222222223,
 'metrics/recall(B)': 1.0,
 'metrics/mAP50(B)': 0.7765919490451477,
 'metrics/mAP50-95(B)': 0.6299769299006207,
 'fitness': 0.6299769299006207}

### Step 7: Data Augmentation using Albumentations

In [ ]:
augment = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.ShiftScaleRotate(
            shift_limit=0.05, scale_limit=0.1, rotate_limit=10, border_mode=0, p=0.5
        ),
    ],
    keypoint_params=A.KeypointParams(format="xy", remove_invisible=False),
)


def yolo_poly_to_px(points, img_w, img_h):
    px = []
    for i in range(0, len(points), 2):
        x = float(points[i]) * img_w
        y = float(points[i + 1]) * img_h
        px.append((x, y))
    return px


def px_poly_to_yolo(points, img_w, img_h):
    out = []
    for x, y in points:
        out.append(x / img_w)
        out.append(y / img_h)
    return out


def build_augmented_dataset_seg(
    input_root="dataset_split",
    output_root="dataset_augmented_seg",
    augmentations_per_image=3,
):
    sets = ["train", "val", "test"]

    if os.path.exists(output_root):
        shutil.rmtree(output_root)

    for s in sets:
        os.makedirs(os.path.join(output_root, s, "images"), exist_ok=True)
        os.makedirs(os.path.join(output_root, s, "labels"), exist_ok=True)

    for split in sets:
        img_dir = os.path.join(input_root, split, "images")
        lbl_dir = os.path.join(input_root, split, "labels")

        out_img_dir = os.path.join(output_root, split, "images")
        out_lbl_dir = os.path.join(output_root, split, "labels")

        images = [
            x
            for x in os.listdir(img_dir)
            if x.lower().endswith((".jpg", ".png", ".jpeg"))
        ]

        print(f"Processing {split}: {len(images)} images")

        for img_name in tqdm(images):
            img_path = os.path.join(img_dir, img_name)
            label_path = os.path.join(lbl_dir, img_name.rsplit(".", 1)[0] + ".txt")

            image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
            h, w = image.shape[:2]

            polygons = []
            labels = []

            with open(label_path, "r") as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    cls = int(parts[0])
                    coords = list(map(float, parts[1:]))
                    poly_px = yolo_poly_to_px(coords, w, h)

                    polygons.append(poly_px)
                    labels.append(cls)

            shutil.copy(img_path, os.path.join(out_img_dir, img_name))
            shutil.copy(
                label_path, os.path.join(out_lbl_dir, img_name.replace(".jpg", ".txt"))
            )

            for i in range(augmentations_per_image):
                keypoints = [p for poly in polygons for p in poly]

                aug = augment(image=image, keypoints=keypoints)
                aug_img = aug["image"]
                aug_kp = aug["keypoints"]

                new_polygons = []
                idx = 0
                for poly in polygons:
                    n = len(poly)
                    new_polygons.append(aug_kp[idx : idx + n])
                    idx += n

                new_txt = []
                for cls, poly in zip(labels, new_polygons):
                    yolo_poly = px_poly_to_yolo(poly, w, h)
                    line = f"{cls} " + " ".join(f"{v:.6f}" for v in yolo_poly)
                    new_txt.append(line)

                new_name = img_name.rsplit(".", 1)[0] + f"_aug{i}.jpg"
                cv2.imwrite(
                    os.path.join(out_img_dir, new_name),
                    cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR),
                )

                with open(
                    os.path.join(out_lbl_dir, new_name.replace(".jpg", ".txt")), "w"
                ) as f:
                    for l in new_txt:
                        f.write(l + "\n")

In [ ]:
build_augmented_dataset_seg(
    input_root="dataset_split",
    output_root="dataset_augmented",
    augmentations_per_image=5,
)

Processing train: 350 images


100%|██████████| 350/350 [00:02<00:00, 155.82it/s]


Processing val: 75 images


100%|██████████| 75/75 [00:00<00:00, 156.11it/s]


Processing test: 75 images


100%|██████████| 75/75 [00:00<00:00, 146.61it/s]


### Step 8: Create data.yaml for Augmented Dataset

In [11]:
data_yaml_aug = {
    "path": str(AUGMENTED_DATA),
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": 1,
    "names": ["fruit"],
}

yaml_path_aug = AUGMENTED_DATA / "data.yaml"
with open(yaml_path_aug, "w") as f:
    yaml.dump(data_yaml_aug, f, default_flow_style=False)

### Step 9: Train YOLOv11 on Augmented Dataset

In [ ]:
model_aug = YOLO("yolo11n.pt")

results_aug = model_aug.train(
    data=str(yaml_path_aug),
    epochs=50,
    imgsz=640,
    batch=16,
    patience=10,
    device=0,
    project=str(BASE_DIR / "runs"),
    name="yolov11_augmented",
    save=True,
    verbose=True,
)

print("✓ Training on augmented dataset completed")

Training YOLOv11 on augmented dataset...
Ultralytics 8.3.235 🚀 Python-3.10.18 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24219MiB)
Ultralytics 8.3.235 🚀 Python-3.10.18 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24219MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_augmented/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, 

In [17]:
best_model = YOLO("runs/yolov11_augmented/weights/best.pt")

test_results = best_model.val(
    data=str(yaml_path), split="test", imgsz=640, batch=16, device=0
)

test_results.results_dict

Ultralytics 8.3.235 🚀 Python-3.10.18 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24219MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1647.2±807.2 MB/s, size: 21.4 KB)
val: Scanning /home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split/test/labels.cache... 75 images, 13 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 75/75 228.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 5/5 8.8it/s 0.6s0.2s
                   all         75         77      0.959      0.961      0.986      0.953
Speed: 1.7ms preprocess, 1.1ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /home/usr4/Arsenii/sbor_data_pfur/lab_7/runs/detect/val9


{'metrics/precision(B)': 0.9590562895433086,
 'metrics/recall(B)': 0.961038961038961,
 'metrics/mAP50(B)': 0.9860309935689857,
 'metrics/mAP50-95(B)': 0.9533627718255001,
 'fitness': 0.9533627718255001}